In [ ]:
# Para manter la maquina conectada cuando no se hace nada
while True:
  True

In [1]:
# Lista de imports
import pandas as pd
# Descargar datos
from google.colab import files
# Stopwords
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
# Para cargar el archivo kaggle.json
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
kaggle.json


In [3]:
# Bases de datos de Kaggle

# DepressionTweets - https://www.kaggle.com/samrats/depressiontweets
!kaggle datasets download -d samrats/depressiontweets
# English depression related posts from Reddit - https://www.kaggle.com/luizfmatos/reddit-english-depression-related-submissions
!kaggle datasets download -d luizfmatos/reddit-english-depression-related-submissions
# Sentimental Analysis NLP - https://www.kaggle.com/ashishpatel26/sentimental-analysis-nlp?select=neg_tweets.txt
!kaggle datasets download -d ashishpatel26/sentimental-analysis-nlp

# Descomprimiendo archivos y eliminado zip
!unzip -q depressiontweets.zip -d .
!rm depressiontweets.zip
!unzip -q sentimental-analysis-nlp.zip -d .
!rm sentimental-analysis-nlp.zip
!unzip -q reddit-english-depression-related-submissions.zip -d .
!rm reddit-english-depression-related-submissions.zip

# Eliminando archivos extras
!rm test_data.csv
!rm train_data.csv

  0% 0.00/3.43M [00:00<?, ?B/s]
100% 3.43M/3.43M [00:00<00:00, 116MB/s]
 96% 17.0M/17.7M [00:00<00:00, 175MB/s]
100% 17.7M/17.7M [00:00<00:00, 163MB/s]
  0% 0.00/577k [00:00<?, ?B/s]
100% 577k/577k [00:00<00:00, 78.4MB/s]


In [4]:
# Si las BD de Kaggle ya se encuentran descargadas

# Obteniendo tablas
temp1 = pd.read_json("_reddit-posts-gatherer-en.submissions_subset.json")
temp2 = pd.read_csv('neg_tweets.txt', header = None, delimiter = "\t")
temp3 = pd.read_csv('pos_tweets.txt', header = None, delimiter = "\t")
temp4 = pd.read_csv('test2Data.csv')
temp5 = pd.read_csv('train2Data.csv')
print("len")
print(len(temp1))
print(len(temp2))
print(len(temp3))
print(len(temp4))
print(len(temp5))

len
32165
1383
617
31000
31000


temp1 - Tiene varias columnas extras. Columna importante: body. Todos los datos son depresivos. De repente tiene emojis.

temp2 y temp 3 - Solo tiene una columna.Todos los datos son depresivos. Hace falta limpiar los datos de @usuario, links. Tiene hashtags.

temp 4 y temp 5 - Tiene dos coulmnas, la del texto y etiquetado. Los textos depresivos y no depresivos estan mezclados.

In [5]:
# Asumiendo que ya se encuentran los archivos temporales (sample_data_1.xlsx y sample_data_2.xlsx) en Colab descargados de https://github.com/shaoxiongji/sw-detection/tree/master/data
temp6 = pd.read_excel("sample_data_1.xlsx")
temp7 = pd.read_excel("sample_data_2.xlsx")
print("len")
print(len(temp6))
print(len(temp7))

# Asumiendo que se corrio el programa Happy_Reddit.ipynb y se subio el archivo obtenido (data_1.txt) en Colab
temp8 = pd.read_csv('data_1.txt', header = None, delimiter = "\t")
print(len(temp8))

len
96
96
9916


Temp 6 - Algunos datos estan duplicados. Columnas importantes: usertext y y. Datos mezclados.

Temp 7 - Algunos datos estan duplicados. Columnas importantes: tweets y y. Datos mezclados.

In [6]:
# Uniendo datos en una sola tabla
table = pd.DataFrame()

# Agregando temp1
cont = len(temp1)
table["text"] = temp1["body"]
# Agregando temp2
cont = cont + len(temp2)
temp2.columns = ["text"]
table = pd.concat([table,temp2])
table["label"] = 1
# Agregando temp3
cont = cont + len(temp3)
temp3.columns = ["text"]
temp3["label"] = 0
table = pd.concat([table,temp3])
# Agregando temp4
cont = cont + len(temp4)
temp4.columns = ["text","label"]
table = pd.concat([table,temp4])
# Agregando temp5
cont = cont + len(temp5)
temp5.columns = ["text","label"]
table = pd.concat([table,temp5])
# Agregando temp6
cont = cont + len(temp6)
temp = temp6[["usertext","y"]]
temp.columns = ["text","label"]
table = pd.concat([table,temp])
# Agregando temp7
cont = cont + len(temp7)
temp7.columns = ["text","label"]
table = pd.concat([table,temp7])
# Agregando temp8
cont = cont + len(temp8)
temp8.columns = ["text"]
temp8["label"] = 0
table = pd.concat([table,temp8])
# Eliminando duplicados
table.drop_duplicates(inplace=True)
# Dar un orden aleatorio de datos
table = table.sample(frac=1)
# Reset index
table.reset_index(drop=True, inplace=True)

print("Total de datos",cont)
print("*********************************************************************")
print(table)
print("*********************************************************************")
print("Total:",len(table))
print("Depress:",len(table[table.label == 1]))
print("Normal:",len(table[table.label == 0]))

Total de datos 106273
*********************************************************************
                                                     text  label
0       meant to be going out for work meal at half an...      1
1       literally no one mentioned him being the best ...      0
2                                         Will i go back?      0
3       He's twice my age (26 F) and he doesn't know w...      1
4       weeks pregnant today and feel so miserable so ...      1
...                                                   ...    ...
102954  I created a website to help people forget thei...      0
102955  love to visit iceland around to see what life ...      0
102956  it am can sleep want to cry and scream and vom...      1
102957  this was so me when ran last sunday you may no...      1
102958  **THE BOTTOM HALF OF THIS POST IS JUST DETAILS...      1

[102959 rows x 2 columns]
*********************************************************************
Total: 102959
Depress: 65105
No

In [7]:
# Descargando dependencias de stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
#Eliminar emojis
!pip install emoji

     |████████████████████████████████| 185 kB 6.6 MB/s 
  Created wheel for emoji: filename=emoji-1.4.1-py3-none-any.whl size=186393 sha256=ce56f76876d83dd29922b9532c8c5c3e92ec04fb45e8a51cc4b1fbedbfd95b30
  Stored in directory: /root/.cache/pip/wheels/8e/68/ac/537456a5331f1405779f2b3c2a578429d2f6d7419e440330d8
Successfully built emoji


In [9]:
#Removiendo emojis
import emoji
import re

def strip_emoji(text):
    #print(emoji.emoji_count(text))
    new_text = re.sub(emoji.get_emoji_regexp(), r"", text)
    return new_text

In [11]:
# Limpiando datos

# Remover URLs
table["text"] = table["text"].str.replace(r"http\S+","")

# Remover menciones de usuarios
table["text"] = table["text"].str.replace(r'(@\w+.*?)',"")

# Remover hashtags
table["text"] = table["text"].str.replace(r'(#\w+.*?)',"")

# Remover emogis
table['text']= table['text'].apply(strip_emoji)

# Cambiar "n't" por " not "
table["text"] = table["text"].str.replace("n't"," not ")

# Stopwords
# Algunas cosas que se podrian quitar de los stop_words
# 'shouldn', 'myself', 'isn', 'me', 'i', 'needn'
# 'mustn', 'didn', 'wasn', 'aren', 'my', 'wouldn', 'doesn', 'ain'
# 'hadn', 'mightn', 'don', 'weren', 'couldn', 'shan', 'hasn'
stop_words = set(stopwords.words('english'))
# Algunas cosas que se podrian agregar a los stopwords
stop_words.update(["'ve", "'s", "'re", "/r/", "'ll", "'d", "'m"])
# Eliminado palabras de negacion
stop_words.difference_update(['no','not','nor','against'])

def remove_stopwords(text):
  word_tokens = word_tokenize(text)
  filtered_sentence = [w.lower() for w in word_tokens if not w.lower() in stop_words]
  return " ".join(filtered_sentence)

table['text'] = table['text'].apply(remove_stopwords)

# table

In [ ]:
# Solo para analizar los textos

# Mostrar tabla
# pd.set_option('display.max_colwidth', None)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     # print(table["text"])
#     for i in range(1000,2000):
#       print(table["text"][i])

In [ ]:
# Ajustando la destribucion
def round_down(num, divisor):
    return num - (num % divisor)

tempD = len(table[table.label == 1])
tempN = len(table[table.label == 0])

if round_down(tempD, 5000) > round_down(tempN, 5000):
  print("Disminuyendo datos depresivos")
  temp = table[table.label == 1]
  table = table[table.label == 0]
  table = pd.concat([table,temp.sample(tempN)])
elif round_down(tempD, 5000) < round_down(tempN, 5000):
  print("Disminuyendo datos normales")
  temp = table[table.label == 0]
  table = table[table.label == 1]
  table = pd.concat([table,temp.sample(tempD)])

# Dar un orden aleatorio de datos
table = table.sample(frac=1)
# Reset index
table.reset_index(drop=True, inplace=True)
print("*********************************************************************")
print(table)
print("*********************************************************************")
print("Total:",len(table))
print("Depress:",len(table[table.label == 1]))
print("Normal:",len(table[table.label == 0]))

Disminuyendo datos depresivos
*********************************************************************
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [ ]:
table.to_csv(r'result.csv',index=False)